In [1]:
import numpy as np
import pandas as pd
import requests, os
import time
import random
import swifter
# import csv
# import datetime as dt
# import json
# import os
# import statistics
from collections import Counter
import matplotlib.pyplot as plt

from fake_useragent import UserAgent
ua = UserAgent()

user_agent = {'User-agent': ua.random}

def get_request(url, parameters=None, rate=60): 
    response = None
    while not response:
        try:
            response = requests.get(url=url, params=parameters, headers=user_agent)
            if not response:
                print('None response. Trying again in {} seconds...'.format(rate))
#                 toggle_vpn()
                time.sleep(rate)
        except:
            print('Request Exception. Trying again in 5 sec...')
            time.sleep(5)
            continue
        
    
    return response.json()

def flatten(t):
    return [item for sublist in t for item in sublist]

In [68]:
df = pd.read_csv('data/combined_cleaned.csv',index_col=0)
df = df[df['owners'] > 1000000][['name','genres']]

df['genres'] = df['genres'].swifter.apply(lambda x: x.lower())

df['genres'] = df['genres'].swifter.apply(lambda x: x.split(', '))

df

Pandas Apply:   0%|          | 0/807 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/807 [00:00<?, ?it/s]

,name,genres
appid,,
10,Counter-Strike,[action]
20,Team Fortress Classic,[action]
30,Day of Defeat,[action]
40,Deathmatch Classic,[action]
50,Half-Life: Opposing Force,[action]
...,...,...
1366540,Dyson Sphere Program,"[indie, simulation, strategy, early access]"
1399780,Spellbreak,"[action, adventure, free to play, rpg]"
1407200,World of Tanks,"[action, free to play, massively multiplayer, ..."


In [69]:
genres_list = flatten(df['genres'].tolist())
genres_set = set(genres_list)
genres_set
delete_genres = ['free to play','indie','violent','gore','early access']
Counter(genres_list).most_common()

[('action', 528),
 ('indie', 288),
 ('adventure', 261),
 ('rpg', 200),
 ('free to play', 179),
 ('strategy', 169),
 ('simulation', 151),
 ('massively multiplayer', 103),
 ('casual', 91),
 ('early access', 44),
 ('sports', 34),
 ('racing', 22),
 ('violent', 2),
 ('gore', 1)]

In [70]:
df['genres'] = df['genres'].swifter.apply(lambda x: [y for y in x if y not in delete_genres])
df = df[df['genres'].map(lambda x: len(x)) != 0]
df

Pandas Apply:   0%|          | 0/807 [00:00<?, ?it/s]

,name,genres
appid,,
10,Counter-Strike,[action]
20,Team Fortress Classic,[action]
30,Day of Defeat,[action]
40,Deathmatch Classic,[action]
50,Half-Life: Opposing Force,[action]
...,...,...
1366540,Dyson Sphere Program,"[simulation, strategy]"
1399780,Spellbreak,"[action, adventure, rpg]"
1407200,World of Tanks,"[action, massively multiplayer, simulation]"


In [98]:
df.loc[244850]

name                     Space Engineers
genres    [action, simulation, strategy]
Name: 244850, dtype: object

In [71]:
df.to_csv('data/shrunkgenres.csv')

In [23]:
import ast

In [27]:
screenshot_df = pd.read_csv('data/screenshots.csv',index_col=0)[['steam_appid','screenshots']]
screenshot_df['screenshots'] = screenshot_df['screenshots'].swifter.apply(lambda x: ast.literal_eval(x))

Pandas Apply:   0%|          | 0/807 [00:00<?, ?it/s]

In [53]:
screenshot_df

,steam_appid,screenshots
0,10,[https://cdn.akamai.steamstatic.com/steam/apps...
1,20,[https://cdn.akamai.steamstatic.com/steam/apps...
2,30,[https://cdn.akamai.steamstatic.com/steam/apps...
3,40,[https://cdn.akamai.steamstatic.com/steam/apps...
4,50,[https://cdn.akamai.steamstatic.com/steam/apps...
...,...,...
802,1366540,[https://cdn.akamai.steamstatic.com/steam/apps...
803,1399780,[https://cdn.akamai.steamstatic.com/steam/apps...
804,1407200,[https://cdn.akamai.steamstatic.com/steam/apps...
805,1426210,[https://cdn.akamai.steamstatic.com/steam/apps...


In [54]:
images_tag = pd.merge(screenshot_df,df,how='right',left_on='steam_appid',right_on='appid')[['appid','screenshots','genres']].set_index('appid')

In [60]:
images_tag = dict(images_tag['screenshots'])

In [56]:
for genre in Counter(genres_list).keys():
    number = len(sum(images_tag['screenshots'][images_tag['genres'].map(lambda genres: genre in genres)].tolist(), []))
    print(genre, number)
    

action 7213
strategy 2293
adventure 3491
rpg 2837
simulation 2484
massively multiplayer 1720
racing 520
casual 1129
sports 666


In [120]:
class_names

array(['sports', 'adventure', 'action', 'simulation',
       'massively multiplayer', 'strategy', 'racing', 'casual', 'rpg'],
      dtype='<U21')

In [114]:
for appid in images_tag.keys():
    for i, image in enumerate(images_tag[appid]):
        response = requests.get(image)
        file = open('image_appid/'+str(appid)+'_'+str(i)+'.png', "wb")
        file.write(response.content)
        file.close()

In [104]:
def label_encode(genres_list):
    return ['1' if x in genres_list else '0' for x in class_names]

id_genre = pd.read_csv('data/shrunkgenres.csv',index_col=0)
id_genre['genres'] = id_genre['genres'].swifter.apply(lambda x: ast.literal_eval(x))
class_names = np.array(list(set(sum(id_genre['genres'].tolist(), []))))

id_genre['genres'] = id_genre['genres'].swifter.apply(label_encode)
id_genre = dict(id_genre['genres'])

Pandas Apply:   0%|          | 0/799 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/799 [00:00<?, ?it/s]

In [105]:
id_genre[244850]

['0', '0', '1', '1', '0', '1', '0', '0', '0']

In [119]:
from pathlib import Path
for path in list(Path('image_appid').glob('*.png')):
    path_str = path.name
    appid = int(path_str.split('_')[0])
    genres = '-'.join(id_genre[appid])
    os.rename('image_appid/'+path.name,'image_appid/'+path.name.replace('.','_'+genres+'.'))

In [107]:
''.join(id_genre[int(next(Path('image_appid').iterdir()).name.split('_')[0])])

'001000000'

In [134]:
next(Path('image_appid').iterdir()).name[-13:-4]

'000000001'

In [136]:
for path in list(Path('image_appid').glob('*.png')):
    path_str = path.name
    genres = path_str[-13:-4]
    beginning = path_str[:-13]
    os.rename('image_appid/'+path.name,'image_appid/'+beginning+'s'.join(genres)+'.png')